# Framework

In [ ]:
import pickle
import pandas as pd
import geopandas as gpd
import numpy as np
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt

from bikewaysim.paths import config
from bikewaysim.framework import bikewaysim_functions
from bikewaysim.impedance_calibration import stochastic_optimization
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs

In [ ]:
_, _, length_dict, geo_dict, _ = rustworkx_routing_funcs.import_calibration_network(config)

## ODS

In [ ]:
# import matched ODS (i.e. already have the correct network nodes)
od_matrix = pd.read_csv(config['bikewaysim_fp']/'od_matrix.csv')
ods = list(set(zip(od_matrix['orig_N'],od_matrix['dest_N'])))

# Network Import

Merge the three network together

In [ ]:
cols_to_keep = ['linkid','reverse_link','travel_time_min','multiplier','link_cost']

current_links = gpd.read_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='current_network').loc[:,cols_to_keep+['geometry']].set_index(['linkid','reverse_link'])
future_links = gpd.read_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='future_network').loc[:,cols_to_keep+['geometry']].set_index(['linkid','reverse_link'])
current_links.columns = 'current_' + current_links.columns
future_links.columns = 'future_' + future_links.columns

links = pd.concat([current_links,future_links],ignore_index=False,axis=1)

merge_these_cols = ['travel_time_min','geometry']

for col in merge_these_cols:
    links[col] = links[f'current_{col}'].fillna(links[f'future_{col}'])
    links.drop(columns=[f'current_{col}',f'future_{col}'],inplace=True)

links.set_geometry('geometry',inplace=True)
links.reset_index(inplace=True)

In [ ]:
improvements = gpd.read_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='improvements')
print(improvements[f"{config['mod_name']}_id"].nunique(),'unique projects')
improvements[improvements[f"{config['mod_name']}_id"].notna()].plot()

# Get link impedance reduction from improvements

Get base link impedance reduction

In [ ]:
# NOTE: we want these to only be negative
links['impedance_reduction'] = links['future_multiplier'] - links['current_multiplier']
links['impedance_reduction'].describe()

In [ ]:
max_impedance_reduction = links[links['impedance_reduction']<0].groupby('linkid')['impedance_reduction'].idxmin()
link_impedance_reduction = links.loc[max_impedance_reduction]
link_impedance_reduction['impedance_reduction_display'] = (link_impedance_reduction['impedance_reduction'].abs() * 100).astype(int)
link_impedance_reduction.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='link_impedance_reduction')

In [ ]:
link_impedance_reduction['impedance_reduction_display']

In [ ]:
undirected_links = links[links['reverse_link']==False]

# Import Routing Results

In [ ]:
# with (config['bikewaysim_fp']/"current_shortest_results.pkl").open('rb') as fh:
#     shortest_results_dict = pickle.load(fh)

# with (config['bikewaysim_fp']/"current_results.pkl").open('rb') as fh:
#     current_results_dict = pickle.load(fh)

# with (config['bikewaysim_fp']/"future_results.pkl").open('rb') as fh:
#     future_results_dict = pickle.load(fh)

with (config['bikewaysim_fp']/'parallel_routing.pkl').open('rb') as fh:
    routing_results = pickle.load(fh)

shortest_results_dict = routing_results['current_traveltime']
current_results_dict = routing_results['current_impedance']
future_results_dict = routing_results['future_impedance']

# Calculate trip travel time/impedance changes

In [ ]:
# #BUG in the travel time dict, there an int stored as an edge instead of the normal (linkid reverse link)
# #current_results_dict.get((67124310.0, 5218791368.0))['edge_list']
# x = []
# for key, item in current_results_dict.items():
#     current_results_dict[key]['edge_list'] = [edge for edge in item['edge_list'] if isinstance(edge,int)==False] 

In [ ]:
travel_time_dict = dict(zip(links['linkid'],links['travel_time_min']))

# get the travel times for each run
current_tt = {key: np.sum([travel_time_dict.get(edge[0],np.nan) for edge in item['edge_list']]) for key, item in current_results_dict.items()}
future_tt = {key: np.sum([travel_time_dict.get(edge[0],np.nan) for edge in item['edge_list']]) for key, item in future_results_dict.items()}
shortest_tt = {key: np.sum([travel_time_dict.get(edge[0],np.nan) for edge in item['edge_list']]) for key, item in shortest_results_dict.items()}

# assign travel times to trips in od_matrix
od_matrix['current_tt_min'] = [current_tt.get(x,np.nan) for x in list((zip(od_matrix['orig_N'],od_matrix['dest_N'])))]
od_matrix['future_tt_min'] = [future_tt.get(x,np.nan) for x in list((zip(od_matrix['orig_N'],od_matrix['dest_N'])))]
od_matrix['shortest_tt_min'] = [shortest_tt.get(x,np.nan) for x in list((zip(od_matrix['orig_N'],od_matrix['dest_N'])))]

# retrieve the trip impedance
current_impedance = {key:item['length'] for key, item in current_results_dict.items()}
future_impedance = {key:item['length'] for key, item in future_results_dict.items()}

# assign trip impedance to trips in od_matrix
od_matrix['current_impedance'] = [current_impedance.get(x,np.nan) for x in list((zip(od_matrix['orig_N'],od_matrix['dest_N'])))]
od_matrix['future_impedance'] = [future_impedance.get(x,np.nan) for x in list((zip(od_matrix['orig_N'],od_matrix['dest_N'])))]

# Impedance Change for trips

Weighted reduction

In [ ]:
od_matrix['impedance_reduction'] = (od_matrix['current_impedance'] - od_matrix['future_impedance'])
maen_reduction = (od_matrix['impedance_reduction'] * od_matrix['trips']).sum() / od_matrix['trips'].sum()
print(f'The weighted average reduction in impedance was {round(maen_reduction,2)} minutes')

In [ ]:
no_reduction = od_matrix[od_matrix['impedance_reduction']==0].shape[0]
print('The impedance for',no_reduction,'OD pairs did not decrease')
print(int(no_reduction/od_matrix.shape[0]*100),'%')

In [ ]:
# distribution isn't too meaningful
# import matplotlib.pyplot as plt
# plt.hist(od_matrix['impedance_reduction'],weights=od_matrix['trips'],bins=20)

In [ ]:
max_reduction = od_matrix['impedance_reduction'].max() #* od_matrix['trips']).sum() / od_matrix['trips'].sum()
print(f"The max reduction in impedance was {round(max_reduction,2)} minutes")

In [ ]:
# some difference if aggregating to the destination taz but it's not too different
# trip_impedance_reduction = od_matrix.groupby('dest_taz').apply(lambda x: ((x.current_impedance-x.future_impedance) * x.trips).sum() / x.trips.sum())
# #studyarea = gpd.read_file(config['studyarea_fp'])
# tazs = gpd.read_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_polygons')#,mask=studyarea)
# tazs['impedance_change'] = tazs['tazid'].map(trip_impedance_reduction)
# tazs.plot('impedance_change')

## Weighted impedance reduction per TAZ

In [ ]:
trip_impedance_reduction = od_matrix.groupby('orig_taz').apply(lambda x: ((x.current_impedance-x.future_impedance) * x.trips).sum() / x.trips.sum())
#studyarea = gpd.read_file(config['studyarea_fp'])
#tazs = gpd.read_file(config['bikewaysim_fp']/'ods.gpkg',layer='trip_origins')#,mask=studyarea)
tazs = gpd.read_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_polygons')#,mask=studyarea)
tazs['impedance_change'] = tazs['tazid'].map(trip_impedance_reduction)
tazs.plot('impedance_change')

In [ ]:
tazs['impedance_change'].isna().sum()

In [ ]:
print(tazs[tazs['impedance_change']==0].shape[0],'TAZs had zero change in impedance')

In [ ]:
tazs[tazs['impedance_change']==0].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='origin_weighted_impedance_nochange')
tazs[tazs['impedance_change']>0].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='origin_weighted_impedance_change')

# Percent Detour

In [ ]:
# od_matrix['detour'] = od_matrix['current_tt_min'] - od_matrix['shortest_tt_min']
# detour = od_matrix.groupby('orig_taz').apply(lambda x: (x.detour * x.trips).sum() / x.trips.sum())
# #od_matrix.hist(od_matrix['detour'],bins=50,weights=od_matrix['trips'])
# tazs['detour'] = tazs['tazid'].map(detour)
# tazs.plot('detour')

In [ ]:
od_matrix['percent_detour'] = ((od_matrix['current_tt_min'] - od_matrix['shortest_tt_min']) / od_matrix['shortest_tt_min'] * 100)#.round(1)
od_matrix['percent_detour'].describe()

In [ ]:
od_matrix['percent_detour'].median()

In [ ]:
x, y = od_matrix.loc[od_matrix['percent_detour'].idxmax()][['orig_N','dest_N']].values

In [ ]:
# for inspecting trips
gpd.GeoDataFrame(
    {'geometry':[geo_dict[x[0]] for x in current_results_dict[(x,y)]['edge_list']]},
    crs=config['projected_crs_epsg']
).explore()

In [ ]:
from matplotlib.ticker import FuncFormatter

def millions(x, pos):
    'The two args are the value and tick position'
    return '%1.1fM' % (x * 1e-6)
formatter = FuncFormatter(millions)

# plt.rcParams['font.family'] = 'Times New Roman'
fig, ax = plt.subplots()
ax.hist(od_matrix['percent_detour'], bins=range(0, 50, 2), alpha=0.3, color='grey', weights=od_matrix['trips'],edgecolor='black')
ax.set_xlabel(f"Percent Detour (%)")
ax.set_ylabel(f"Number of Trips (N = {od_matrix['trips'].sum() / 10**6 :.1f} Million)")

mean_percent_detour = (od_matrix['percent_detour'] * od_matrix['trips']).sum() / od_matrix['trips'].sum()
ax.axvline(mean_percent_detour, color='red', linestyle='dashed', linewidth=1)
ax.text(od_matrix['percent_detour'].mean() * 1.5, plt.gca().get_ylim()[1] * 0.9, f"Mean: {mean_percent_detour:,.1f}% detour", color='red', ha='left')

ax.xaxis.set_major_locator(MultipleLocator(10))
ax.xaxis.set_minor_locator(MultipleLocator(2))
ax.yaxis.set_major_locator(MultipleLocator(0.5*10**6))
ax.yaxis.set_minor_locator(MultipleLocator(0.1*10**6))
ax.yaxis.set_major_formatter(formatter)

plt.savefig(config['figures_fp']/'percent_detour.png',dpi=300)
plt.show()

In [ ]:
percent_detour = od_matrix.groupby('orig_taz').apply(lambda x: (x.percent_detour * x.trips).sum() / x.trips.sum())
percent_detour.describe()

In [ ]:
# tazs = gpd.read_file(config['bikewaysim_fp']/'ods.gpkg',layer='trip_origins')
tazs = gpd.read_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_polygons')#,mask=studyarea)
tazs['percent_detour'] = tazs['tazid'].map(percent_detour)
tazs.plot('percent_detour')

In [ ]:
tazs.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='percent_detour')

## Percent Detour Change Due to Network Improvements (future)

In [ ]:
od_matrix['improved_percent_detour'] = ((od_matrix['future_tt_min'] - od_matrix['shortest_tt_min']) / od_matrix['shortest_tt_min'] * 100)#.round(1)
improved_percent_detour = od_matrix.groupby('orig_taz').apply(lambda x: (x.improved_percent_detour * x.trips).sum() / x.trips.sum())
improved_percent_detour.describe()

In [ ]:
# tazs['improved_percent_detour'] = tazs['tazid'].map(improved_percent_detour)
# tazs['percent_detour_change'] = tazs['improved_percent_detour'] - tazs['percent_detour']
# tazs['percent_detour_change'].hist()

In [ ]:
# tazs.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='percent_detour')

# Link Betweenness Centrality (LBC)
We need this to visualize how trips shifted and to detemrine the impedance reduction contribution of each project.

In [ ]:
#directed is number of trips by direction, undirected is combined number of trips
shortest_undirected_btw_centrality = {key:0 for key in links['linkid'].tolist()}
shortest_directed_btw_centrality = {key:0 for key in list(zip(links['linkid'],links['reverse_link']))}
current_undirected_btw_centrality = {key:0 for key in links['linkid'].tolist()}
current_directed_btw_centrality = {key:0 for key in list(zip(links['linkid'],links['reverse_link']))}
future_undirected_btw_centrality = {key:0 for key in links['linkid'].tolist()}
future_directed_btw_centrality = {key:0 for key in list(zip(links['linkid'],links['reverse_link']))}

dict_keys = list(set.union(set(current_results_dict.keys()),set(future_results_dict.keys())))
for od_pair in dict_keys:
    #get number of trips for that origin node and destinnation node
    o_check = od_matrix['orig_N']==od_pair[0]
    d_check = od_matrix['dest_N']==od_pair[1]
    num_trips = od_matrix.loc[o_check & d_check,'trips'].sum()
    
    #retreive the edge list
    shortest_modeled_edges = shortest_results_dict.get(od_pair,0)
    current_modeled_edges = current_results_dict.get(od_pair,0)
    future_modeled_edges = future_results_dict.get(od_pair,0)
    
    #current
    if shortest_modeled_edges != 0:
        #retrieve the edge list
        shortest_modeled_edges = [tuple(x) for x in shortest_modeled_edges['edge_list']]
        #now add the num trips for each edge in the edge list
        for edge in shortest_modeled_edges:
            shortest_directed_btw_centrality[edge] += num_trips
            shortest_undirected_btw_centrality[edge[0]] += num_trips

    #current
    if current_modeled_edges != 0:
        #retrieve the edge list
        current_modeled_edges = [tuple(x) for x in current_modeled_edges['edge_list']]
        #now add the num trips for each edge in the edge list
        for edge in current_modeled_edges:
            current_directed_btw_centrality[edge] += num_trips
            current_undirected_btw_centrality[edge[0]] += num_trips
    #future
    if future_modeled_edges != 0:
        #retrieve the edge list
        future_modeled_edges = [tuple(x) for x in future_modeled_edges['edge_list']]
        #now add the num trips for each edge in the edge list
        for edge in future_modeled_edges:
            future_directed_btw_centrality[edge] += num_trips
            future_undirected_btw_centrality[edge[0]] += num_trips

#only keep non_zero entries
shortest_undirected_btw_centrality = {key:item for key,item in shortest_undirected_btw_centrality.items() if item > 0}
shortest_directed_btw_centrality = {key:item for key,item in shortest_directed_btw_centrality.items() if item > 0}
current_undirected_btw_centrality = {key:item for key,item in current_undirected_btw_centrality.items() if item > 0}
current_directed_btw_centrality = {key:item for key,item in current_directed_btw_centrality.items() if item > 0}
future_undirected_btw_centrality = {key:item for key,item in future_undirected_btw_centrality.items() if item > 0}
future_directed_btw_centrality = {key:item for key,item in future_directed_btw_centrality.items() if item > 0}            

#turn to dataframe
# directed btw centrality useful for directed bike facilities
df0 = pd.DataFrame.from_dict(shortest_directed_btw_centrality,orient='index',columns=['shortest_btw_centrality'])
df1 = pd.DataFrame.from_dict(current_directed_btw_centrality,orient='index',columns=['current_btw_centrality'])
df2 = pd.DataFrame.from_dict(future_directed_btw_centrality,orient='index',columns=['future_btw_centrality'])
directed_btw_centrality = pd.concat([df0,df1,df2],ignore_index=False,axis=1)
directed_btw_centrality.fillna(0,inplace=True)

df0 = pd.DataFrame.from_dict(shortest_undirected_btw_centrality,orient='index',columns=['shortest_btw_centrality'])
df1 = pd.DataFrame.from_dict(current_undirected_btw_centrality,orient='index',columns=['current_btw_centrality'])
df2 = pd.DataFrame.from_dict(future_undirected_btw_centrality,orient='index',columns=['future_btw_centrality'])
undirected_btw_centrality = pd.concat([df0,df1,df2],ignore_index=False,axis=1)
undirected_btw_centrality.fillna(0,inplace=True)
undirected_btw_centrality.reset_index(inplace=True)
undirected_btw_centrality.rename(columns={'index':'linkid'},inplace=True)

#calculate the change
directed_btw_centrality['impedance_change'] = directed_btw_centrality['current_btw_centrality'] - directed_btw_centrality['shortest_btw_centrality']
undirected_btw_centrality['impedance_change'] = undirected_btw_centrality['current_btw_centrality'] - undirected_btw_centrality['shortest_btw_centrality']

directed_btw_centrality['improvement_change'] = directed_btw_centrality['future_btw_centrality'] - directed_btw_centrality['current_btw_centrality']
undirected_btw_centrality['improvement_change'] = undirected_btw_centrality['future_btw_centrality'] - undirected_btw_centrality['current_btw_centrality']

#and pct change
#undirected_btw_centrality['pct_change'] = undirected_btw_centrality['change'] / undirected_btw_centrality['current_btw_centrality'] * 100


In [ ]:
#add geometry data
undirected_btw_centrality_gdf = pd.merge(undirected_btw_centrality,undirected_links,on='linkid')
undirected_btw_centrality_gdf = gpd.GeoDataFrame(undirected_btw_centrality_gdf,geometry='geometry',crs=config['projected_crs_epsg'])

Plot change in LBC going from minimum travel time LBC to impedance LBC (both on current network)

In [ ]:
undirected_btw_centrality_gdf[undirected_btw_centrality_gdf['impedance_change']!=0].plot('impedance_change')

Plot change in LBC going from impednace LBC on current network to impedance LBC on future network

In [ ]:
undirected_btw_centrality_gdf[undirected_btw_centrality_gdf['improvement_change']!=0].plot('improvement_change')

In [ ]:
undirected_btw_centrality_gdf.describe()

Export GPKGs for visualization in QGIS

In [ ]:
#travel time lbc
undirected_btw_centrality_gdf[undirected_btw_centrality_gdf['shortest_btw_centrality']!=0].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='lbc_shortest')
undirected_btw_centrality_gdf[undirected_btw_centrality_gdf['current_btw_centrality']!=0].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='lbc_current')
undirected_btw_centrality_gdf[undirected_btw_centrality_gdf['future_btw_centrality']!=0].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='lbc_future')

undirected_btw_centrality_gdf.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='lbc_impedance_change')
undirected_btw_centrality_gdf.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='lbc_improvement_change')

## Impedance Reduction Contribution 

### Multiply Link Impedance Reduction by Betweenness Centrality for the Improved Links

$ Z_{i}=\frac{\sum_{l_{a}}{d}}{3} $

In [ ]:
undirected_btw_centrality_gdf['impedance_contribution'] = undirected_btw_centrality_gdf.length * undirected_btw_centrality_gdf['impedance_reduction'] * undirected_btw_centrality_gdf['future_btw_centrality']
#undirected_btw_centrality_gdf['length'] = undirected_btw_centrality_gdf.length

In [ ]:
undirected_btw_centrality_gdf.columns

In [ ]:
undirected_btw_centrality_gdf['impedance_contribution'].min()

In [ ]:
undirected_btw_centrality_gdf['impedance_contribution'].max()

In [ ]:
# #bring in name
# facility_names = gpd.read_file(config['bicycle_facilities_fp']/'network_modifications.gpkg',layer=config['mod_name'],ignore_geometry=True)
# facility_names = facility_names[[config['mod_name']+'_id',config['mod_name']+'_name',config['mod_name']+'_facilitytype']]
# impedance_contribution.crs

In [ ]:
#merge with df
impedance_contribution = pd.merge(undirected_btw_centrality_gdf,improvements.drop(columns=['geometry']),on='linkid')
impedance_contribution['Length (mi)'] = impedance_contribution.length / 5280
impedance_contribution = impedance_contribution[['geometry','impedance_contribution','coa_name', 'improvement','Length (mi)']]
impedance_contribution = impedance_contribution.dissolve(['coa_name','improvement'],aggfunc=[np.sum]).sort_values(('impedance_contribution', 'sum')).reset_index()
impedance_contribution.columns = ['Name','Facility Type','geometry','Impedance Contribution','Length (mi)']
impedance_contribution['Impedance Contribution per Mile'] = impedance_contribution['Impedance Contribution'] / impedance_contribution['Length (mi)']

In [ ]:
impedance_contribution

In [ ]:
impedance_contribution.sort_values('Impedance Contribution per Mile',inplace=True)
impedance_contribution['Impedance Contribution per Mile'] = impedance_contribution['Impedance Contribution per Mile'].abs().round(2)
impedance_contribution['Impedance Contribution'] = impedance_contribution['Impedance Contribution'].abs().round(2)
impedance_contribution['Length (mi)'] = impedance_contribution['Length (mi)'].round(3)
impedance_contribution.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='impedance_contribution')

In [ ]:
impedance_contribution

In [ ]:
impedance_contribution.plot('Impedance Contribution per Mile')

Find top 10 facilities that are longer than 10 feet

In [ ]:
top_ten = impedance_contribution#[impedance_contribution['Length (mi)']]
top_ten = top_ten.iloc[0:10]
top_ten

In [ ]:
i = -1

In [ ]:
i += 1
top_ten.iloc[[i]].explore()


In [ ]:
# # manaully type out the descriptive names
# descriptive_names = [
#     'Pryor St to Piedmont Ave',
#     'To Highland Ave',
#     'High',
#     'MLK Jr Dr SE to North Ave NE',
#     'North Ave NE to 14th St NE',
#     'Mitchell St SW to MLK Jr Drive SW',
#     'Alabama St SW to Walton St NW',
#     'Memorial Dr SW to Mitchell St SW',
#     'North Ave NE to 15th St NE',
#     'Whitehall Terr to RDA Fwy'
# ]
top_ten['Segment'] = 'placeholder'#descriptive_names


In [ ]:
top_ten.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='top_ten_infra')

In [ ]:
#turn to str to add commas
top_ten.reset_index(inplace=True)


In [ ]:
# top_ten[['Impedance Contribution','Impedance Contribution per Mile','Length (mi)']] = top_ten[['Impedance Contribution','Impedance Contribution per Mile','Length (mi)']].applymap(lambda x: f"{x:,.1f}")
top_ten = top_ten[['Name','Segment','Facility Type','Impedance Contribution per Mile','Impedance Contribution','Length (mi)']]

In [ ]:
top_ten['Impedance Contribution'] = top_ten['Impedance Contribution'] / 10**6
top_ten['Impedance Contribution per Mile'] = top_ten['Impedance Contribution per Mile'] / 10**6
top_ten.to_csv(config['bikewaysim_fp']/'impedance_contribution.csv',index=False)

In [ ]:
impedance_contribution.explore(popup=True)

In [ ]:
# bike_facility = undirected_btw_centrality_gdf[undirected_btw_centrality_gdf[f"{config['mod_name']}_id"].notna()]
# bike_facility_geo = bike_facility.dissolve(f"{config['mod_name']}_id")[['geometry']]
# impedance_contribution = bike_facility.groupby(f"{config['mod_name']}_id")[['impedance_contribution','length']].sum().abs()
# impedance_contribution = pd.concat([impedance_contribution,bike_facility_geo],axis=1,ignore_index=False)
# impedance_contribution['impedance_contribution_per_ft'] = impedance_contribution['impedance_contribution'] / impedance_contribution['length']
# impedance_contribution = gpd.GeoDataFrame(impedance_contribution,crs=config['projected_crs_epsg'])
# impedance_contribution.plot('impedance_contribution_per_ft')

In [ ]:
# impedance_contribution = impedance_contribution[['impedance_contribution_per_ft','impedance_contribution','length']]
# impedance_contribution.reset_index(inplace=True)
# impedance_contribution['impedance_contribution_per_ft'] = impedance_contribution['impedance_contribution_per_ft'].round(2)
# impedance_contribution['impedance_contribution'] = impedance_contribution['impedance_contribution'].round(2)
# impedance_contribution['length'] = impedance_contribution['length'].round(0)
# impedance_contribution.sort_values('impedance_contribution_per_ft',ascending=False,inplace=True)

# Bikesheds

In [ ]:
# import each network
with (config['bikewaysim_fp']/'current_traveltime_novirtual.pkl').open('rb') as fh:
    current_traveltime_G = pickle.load(fh)
with (config['bikewaysim_fp']/'current_impedance_novirtual.pkl').open('rb') as fh:
    current_impedance_G = pickle.load(fh)
with (config['bikewaysim_fp']/'future_impedance_novirtual.pkl').open('rb') as fh:
    future_impedance_G = pickle.load(fh)

In [ ]:
# get point start/end node gometries from the links
from shapely.ops import Point
matching_network = links.copy()
matching_network.geometry = matching_network[['geometry','reverse_link']].apply(lambda x: Point(list(x.geometry.coords)[0]) if x.reverse_link == False else Point(list(x.geometry.coords)[-1]),axis=1)
# matching_network = matching_network[['geometry']]

In [ ]:
from importlib import reload
reload(bikewaysim_functions)
from shapely.ops import Point
# inputs
limit = 10 # minutes
coordinates = (-84.380797,33.780729)#(-84.381260,33.779508)#(-84.392086,33.750401)
coordinates_gdf = gpd.GeoDataFrame({'geometry':Point(coordinates)},index=[0],crs='epsg:4326')
coordinates_gdf.to_crs(links.crs)

In [ ]:
current_traveltime_bikeshed = bikewaysim_functions.make_bikeshed_rx(coordinates,limit,matching_network,links,current_traveltime_G)
current_impedance_bikeshed = bikewaysim_functions.make_bikeshed_rx(coordinates,limit,matching_network,links,current_impedance_G)
future_impedance_bikeshed = bikewaysim_functions.make_bikeshed_rx(coordinates,limit,matching_network,links,future_impedance_G)

# removed
current_traveltime_bikeshed_li = set([tuple(x) for x in current_traveltime_bikeshed[['linkid','reverse_link']].values])
current_impedance_bikeshed_li = set([tuple(x) for x in current_impedance_bikeshed[['linkid','reverse_link']].values])
future_impedance_bikeshed_li = set([tuple(x) for x in future_impedance_bikeshed[['linkid','reverse_link']].values])
removed = current_traveltime_bikeshed_li - current_impedance_bikeshed_li
added = current_impedance_bikeshed_li - current_traveltime_bikeshed_li
new = future_impedance_bikeshed_li - current_impedance_bikeshed_li

links.set_index(['linkid','reverse_link'],inplace=True)
removed = links.loc[removed]
added= links.loc[added]
new = links.loc[new]
links.reset_index(inplace=True)


In [ ]:
# current_traveltime_bikeshed[current_traveltime_bikeshed['linkid'].duplicated()==False].explore()

In [ ]:
print('Removed',removed.shape[0])
print('Added',added.shape[0])
print('New',new.shape[0])

In [ ]:
coordinates_gdf.to_file(config['bikewaysim_fp']/'new_bikesheds.gpkg',layer='center')
current_traveltime_bikeshed.to_file(config['bikewaysim_fp']/'new_bikesheds.gpkg',layer='traveltime')
current_impedance_bikeshed.to_file(config['bikewaysim_fp']/'new_bikesheds.gpkg',layer='current')
future_impedance_bikeshed.to_file(config['bikewaysim_fp']/'new_bikesheds.gpkg',layer='future')
removed.to_file(config['bikewaysim_fp']/'new_bikesheds.gpkg',layer='removed')
added.to_file(config['bikewaysim_fp']/'new_bikesheds.gpkg',layer='added')
new.to_file(config['bikewaysim_fp']/'new_bikesheds.gpkg',layer='new')

In [ ]:
#create a bikeshed by going through each impedance column
for impedance in impedances:
    bikeshed[impedance], bikeshed_node = make_bikeshed(links,nodes,taz,ods,radius,buffer_size,impedance)
    bikeshed[impedance].to_file(dir_fp/f'{impedance}/bikesheds.gpkg',layer='bikeshed')

    #find removed links (can no longer be reached)
    rem_links = bikeshed['mins'][-(bikeshed['mins']['A_B'].isin(bikeshed[impedance]['A_B']))]
    if rem_links.shape[0] > 0:
        rem_links.to_file(dir_fp/f'{impedance}/bikesheds.gpkg',layer='removed')

    #find added links (can now be reached)
    add_links = bikeshed[impedance][-bikeshed[impedance]['A_B'].isin(bikeshed['mins']['A_B'])]
    if add_links.shape[0] > 0:
        add_links.to_file(dir_fp/f'{impedance}/bikesheds.gpkg',layer='add')

In [ ]:
#for the improved network
for impedance in impedances:
    bikeshed['improved' + impedance], _ = make_bikeshed(improved_links,nodes,taz,ods,radius,buffer_size,impedance)
    bikeshed['improved' + impedance].to_file(dir_fp/f'{impedance}/bikesheds.gpkg',layer='improved_bikeshed')

    #find added links from improvement
    add_impr = bikeshed['improved'+impedance]
    add_impr = add_impr[-add_impr['A_B'].isin(bikeshed[impedance]['A_B'])]
    if add_impr.shape[0] > 0:
        add_impr.to_file(dir_fp/f'{impedance}/bikesheds.gpkg',layer='improvment add')
    else:
        print('No new links added')

## Unfinished past here

Merge improvement with the cycling faciliity geodataframe

In [ ]:
new_facilities = gpd.read_file(config['bicycle_facilities_fp']/'network_modifications.gpkg',layer=config['mod_name'])
new_facilities = new_facilities[[f"{config['mod_name']}_id",f"{config['mod_name']}_name",f"{config['mod_name']}_osm_type"]]
impedance_contribution = pd.merge(impedance_contribution,new_facilities,on=f"{config['mod_name']}_id")
#impedance_contribution = impedance_contribution[[f"{config['mod_name']}_name",f"{config['mod_name']}_osm_type",'impedance_contribution_per_ft','impedance_contribution', 'length']]

Format and export csv

In [ ]:
impedance_contribution.head()

In [ ]:
impedance_contribution.to_csv(config['bikewaysim_fp']/'impedance_contribution.csv',index=False)
impedance_contribution.head(10)

In [ ]:
impedance_contribution['impedance_contribution_per_ft'] = impedance_contribution['impedance_contribution_per_ft'].astype(int)
impedance_contribution['length'] = impedance_contribution['length'].astype(int)


In [ ]:
impedance_contribution = bike_facility.groupby(f"config['mod_id']}")[['impedance_contribution','length']].sum().abs()
#impedance_contribution = (impedance_contribution['impedance_contribution'] / impedance_contribution['length']).sort_values()

In [ ]:
impedance_contribution

In [ ]:
undirected_btw_centrality_gdf['improvement_impedance_reduction'] = undirected_btw_centrality_gdf['config['mod_id']'].map(test)

In [ ]:
undirected_btw_centrality_gdf[undirected_btw_centrality_gdf['config['mod_id']'].notna()].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='improvement_impedance_reduction')
undirected_btw_centrality_gdf[undirected_btw_centrality_gdf['config['mod_id']'].isna()]

In [ ]:

#import the ones with names too
improvements

In [ ]:
improvements[['facility_fwd_new','config['mod_id']']].drop_duplicates()

In [ ]:
test['impedance_reduction'] (undirected_btw_centrality_gdf['future_link_cost'] - undirected_btw_centrality_gdf['current_link_cost'])

In [ ]:
undirected_links

In [ ]:
improvements.head()

In [ ]:
undirected_btw_centrality_gdf.head()

In [ ]:
test = pd.merge(test,facility_ids,on='linkid',how='left')

In [ ]:
test['impedance_change_cont'] = test['counts'] * test['impedance_change']

In [ ]:
#import the coa id back in so that we can aggregate by the faciility
facility_ids = gpd.read_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer='coa')[['config['mod_id']','linkid']]

In [ ]:
result = test[test['impedance_change_cont']>0].groupby(['config['mod_id']','improvement'])['impedance_change_cont'].sum().sort_values(ascending=False)
result.reset_index(inplace=True)

In [ ]:
facility_ids = gpd.read_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer='coa')
facility_ids = facility_ids.dissolve('config['mod_id']').reset_index()

In [ ]:
facility_ids = pd.merge(facility_ids,result,on='config['mod_id']')
facility_ids.explore('impedance_change_cont')

In [ ]:
results_dict

In [ ]:
#od_matrix['current_impedance'] = 
results_dict.apply(lambda x: current_results_dict.get((x['origin_N'],),axis=1)

In [ ]:
# find the impedance for each trip

In [ ]:
results_dict[(6846937343, 5416049647)].keys()

In [ ]:
#figure out why this is na
pd.Series([item['length'] for key, item in results_dict.items()]).isna().sum()

In [ ]:
pd.Series([item['length'] for key, item in results_dict.items()]).isna().sum()

In [ ]:
len(results_dict)

In [ ]:
# get the results and assigned to ods (store edge sequence in a dict soemwehre)
ods.map(results_dict)

In [ ]:
# calculate btwn centrality



In [ ]:
# import test case/ base case (think of a better word) network
with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links,turns = pickle.load(fh)
# import the calibration results
with (config['calibration_fp']/"calibration_results.pkl").open('rb') as fh:
    calibration_results = pickle.load(fh)
turn_G = modeling_turns.make_turn_graph(turns)